In [ ]:
## Manually Upload WeatherData20years.csv to Colab

## Setup 
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
from datetime import timedelta
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from IPython.display import Image
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, roc_curve, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from mlxtend.preprocessing import TransactionEncoder
from sklearn.model_selection import GridSearchCV
from yellowbrick.features import rank2d, rank1d
import statsmodels.api as sm
from statsmodels.tools import add_constant
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import InterclusterDistance
from yellowbrick.cluster import SilhouetteVisualizer
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier



%matplotlib inline
plt.style.use('seaborn-white')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WeatherData20years.csv to WeatherData20years.csv


In [ ]:
import io
weather = pd.read_csv(io.BytesIO(uploaded['WeatherData20years.csv']))

In [ ]:
weather.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,946684800,2000-01-01 00:00:00 +0000 UTC,-25200,Colorado Springs,38.833882,-104.821363,278.81,276.05,275.72,280.51,...,100,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
1,946688400,2000-01-01 01:00:00 +0000 UTC,-25200,Colorado Springs,38.833882,-104.821363,277.48,275.17,275.24,278.50,...,110,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
2,946692000,2000-01-01 02:00:00 +0000 UTC,-25200,Colorado Springs,38.833882,-104.821363,276.64,275.37,274.87,279.10,...,170,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
3,946695600,2000-01-01 03:00:00 +0000 UTC,-25200,Colorado Springs,38.833882,-104.821363,271.65,269.68,269.60,275.20,...,100,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
4,946699200,2000-01-01 04:00:00 +0000 UTC,-25200,Colorado Springs,38.833882,-104.821363,271.26,268.36,269.44,274.60,...,30,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n


In [ ]:
weather.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level',
       'grnd_level', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [ ]:
weather = weather.drop(['dt','timezone','city_name','lat','lon','weather_id','weather_icon','sea_level','grnd_level',
                        'rain_3h','snow_3h','weather_main'], axis =1)

In [ ]:
weather.columns

Index(['dt_iso', 'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'snow_1h',
       'clouds_all', 'weather_description'],
      dtype='object')

In [ ]:
#High Colinearity with these features and "temp" which we kept 
weather = weather.drop(['feels_like','temp_min','temp_max'], axis = 1)

In [ ]:
weather['dt_iso'] = weather['dt_iso'].str.split('+').str.get(0)
weather['dt_iso'] = pd.to_datetime(weather['dt_iso'])

In [ ]:
years = [2015, 2016, 2017, 2018, 2019]
weather = weather[weather['dt_iso'].dt.year.isin(years)]

In [ ]:
weather.head()

,dt_iso,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,clouds_all,weather_description
138552,2015-01-01 00:00:00,265.96,1020,48,4.6,150,NaN,NaN,20,few clouds
138553,2015-01-01 01:00:00,267.38,1019,57,2.1,190,NaN,NaN,1,sky is clear
138554,2015-01-01 02:00:00,267.50,1019,54,3.1,180,NaN,NaN,1,sky is clear
138555,2015-01-01 03:00:00,265.15,1019,66,1.0,300,NaN,NaN,1,sky is clear
138556,2015-01-01 04:00:00,265.08,1019,68,0.5,160,NaN,NaN,1,sky is clear


In [ ]:
# Kelvin to F
weather['temp'] = (weather['temp'] - 273.15) * 9/5 + 32

In [ ]:
weather.head()

,dt_iso,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,clouds_all,weather_description
138552,2015-01-01 00:00:00,19.058,1020,48,4.6,150,NaN,NaN,20,few clouds
138553,2015-01-01 01:00:00,21.614,1019,57,2.1,190,NaN,NaN,1,sky is clear
138554,2015-01-01 02:00:00,21.830,1019,54,3.1,180,NaN,NaN,1,sky is clear
138555,2015-01-01 03:00:00,17.600,1019,66,1.0,300,NaN,NaN,1,sky is clear
138556,2015-01-01 04:00:00,17.474,1019,68,0.5,160,NaN,NaN,1,sky is clear


In [ ]:
#One-Hot Encoding
le = preprocessing.LabelEncoder()
weather['weather_description'] = le.fit_transform(weather['weather_description'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
weatherlabel = pd.Series(le_name_mapping, name='Weather Label')
print(weatherlabel)

broken clouds                    0
drizzle                          1
dust                             2
extreme rain                     3
few clouds                       4
fog                              5
haze                             6
heavy intensity drizzle          7
heavy intensity rain             8
heavy intensity shower rain      9
heavy shower snow               10
heavy snow                      11
light intensity drizzle         12
light intensity shower rain     13
light rain                      14
light rain and snow             15
light shower sleet              16
light shower snow               17
light snow                      18
mist                            19
moderate rain                   20
overcast clouds                 21
proximity thunderstorm          22
ragged thunderstorm             23
scattered clouds                24
shower rain                     25
shower sleet                    26
shower snow                     27
sky is clear        

In [ ]:
weather.head()

,dt_iso,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,clouds_all,weather_description
138552,2015-01-01 00:00:00,19.058,1020,48,4.6,150,NaN,NaN,20,4
138553,2015-01-01 01:00:00,21.614,1019,57,2.1,190,NaN,NaN,1,28
138554,2015-01-01 02:00:00,21.830,1019,54,3.1,180,NaN,NaN,1,28
138555,2015-01-01 03:00:00,17.600,1019,66,1.0,300,NaN,NaN,1,28
138556,2015-01-01 04:00:00,17.474,1019,68,0.5,160,NaN,NaN,1,28


In [ ]:
weather.to_csv('Weather_cleaned.csv')